In [ ]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import Lasso

def omp_manual(dictionary, signal, sparsity):
    n_features = dictionary.shape[1]
    # print(n_features)
    residual = signal
    idx = []
    coefficients = np.zeros(n_features)

    for _ in range(sparsity):

        # Step 1: Find the best matching atom (column from the dictionary)
        projections = dictionary.T @ residual
        best_atom = np.argmax(projections)
        # print(f'best atoms : {best_atom}')
        # Step 2: Add the index of the best atom to the list
        idx.append(best_atom)
        # print(f'idx : {idx}')
        
        # Step 3: Solve least squares to find new coefficients
        selected_atoms = dictionary[:, idx]
        # print(f'selected_atoms : {selected_atoms}')
        coefficients_ls, _, _, _ = np.linalg.lstsq(selected_atoms, signal, rcond=None)
        # print(f'coefficients_ls : {coefficients_ls}')
        # Step 4: Update the residual
        residual = signal - selected_atoms @ coefficients_ls

    # Store the final coefficients
    for i, index in enumerate(idx):
        coefficients[index] = coefficients_ls[i]

    return coefficients

def lasso_method(dictionary, signal, alpha=0.01):
    lasso = Lasso(alpha=alpha, fit_intercept=False)
    lasso.fit(dictionary, signal)
    return lasso.coef_


# Example usage
dictionary = np.random.rand(10, 15)  # Dictionary of 15 atoms, each of length 10
signal = np.random.rand(10)           # Input signal of length 10
sparsity = 3                          # Desired sparsity level
print(dictionary.shape)
# print(signal.shape)
coefficients = omp_manual(dictionary, signal, sparsity)
print("Extracted Coefficients:", coefficients)


coefficients_lasso = lasso_method(dictionary, signal)
print("Extracted Coefficients (Lasso):", coefficients_lasso)


def find_sparse_coefficients(tSample, D, n_nonzero_coefs=3):
    print(f'Dictionary: {D.shape}')
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_
print(f'Other way coefficient: {find_sparse_coefficients(signal, dictionary)}')

(10, 15)
Extracted Coefficients: [0.36705987 0.2818796  0.         0.         0.         0.
 0.         0.         0.         0.         0.49342259 0.
 0.         0.         0.        ]
Extracted Coefficients (Lasso): [ 0.26192678  0.23273936 -0.         -0.          0.00980209  0.
 -0.25619944 -0.          0.          0.35855601  0.47205512  0.
 -0.02809427  0.01721322  0.        ]
Dictionary: (10, 15)
Other way coefficient: [0.44320027 0.28095072 0.         0.         0.         0.
 0.         0.         0.         0.         0.59095459 0.
 0.         0.         0.        ]
